In [20]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem.lancaster import LancasterStemmer
import pyLDAvis
import pyLDAvis.sklearn
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

C:\Users\wuyim\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [16]:
df=pd.read_csv('RFC.csv').dropna(axis=0,how='any')
df.drop(columns=['Unnamed: 0','title'],inplace=True)
df_inf=pd.read_csv('rfc_infor.csv')
df_inf.drop(columns='Unnamed: 0',inplace=True)
df_dns=df_inf[df_inf.area2=='DNS'].copy()
df_dns_full=pd.merge(df_dns,df,on='num',how='left')
df_dns_full.head()

,num,title,date,status,stream,area,wg,area2,key words,content
0,8624,Algorithm Implementation Requirements and Usag...,June 2019,PROPOSED STANDARD,IETF,ops,dnsop,DNS,NaN,\n\n\n\n\n\nInternet Engineering Task Force (I...
1,8586,Loop Detection in Content Delivery Networks (C...,April 2019,PROPOSED STANDARD,IETF,art,httpbis,DNS,"CDN-Loop,",\n\n\n\n\n\nInternet Engineering Task Force (I...
2,8567,Customer Management DNS Resource Records,1 April 2019,INFORMATIONAL,INDEPENDENT,NaN,NaN,DNS,NaN,\n\n\n\n\n\nIndependent Submission ...
3,8553,DNS Attrleaf Changes: Fixing Specifications Th...,March 2019,BEST CURRENT PRACTICE,IETF,ops,dnsop,DNS,"DNS, Domain Name System",\n\n\n\n\n\nInternet Engineering Task Force (I...
4,8552,Scoped Interpretation of DNS Resource Records ...,March 2019,BEST CURRENT PRACTICE,IETF,ops,dnsop,DNS,"DNS, Domain Name System",\n\n\n\n\n\nInternet Engineering Task Force (I...


In [33]:
stemmer = SnowballStemmer("english")
def lemmatize_stemming(text):
    #return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
    return text
def preprocess(text):
    result = []
    blank=' '
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) >= 3:
            result.append(lemmatize_stemming(token))
            
    return blank.join(result)
    

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]])) 
    print()

df_dns_full['cutted_words']=df_dns_full.content.apply(preprocess)
df_dns_full.head()

,num,title,date,status,stream,area,wg,area2,key words,content,cutted_words
0,8624,Algorithm Implementation Requirements and Usag...,June 2019,PROPOSED STANDARD,IETF,ops,dnsop,DNS,NaN,\n\n\n\n\n\nInternet Engineering Task Force (I...,internet engineering task force ietf wouters r...
1,8586,Loop Detection in Content Delivery Networks (C...,April 2019,PROPOSED STANDARD,IETF,art,httpbis,DNS,"CDN-Loop,",\n\n\n\n\n\nInternet Engineering Task Force (I...,internet engineering task force ietf ludin req...
2,8567,Customer Management DNS Resource Records,1 April 2019,INFORMATIONAL,INDEPENDENT,NaN,NaN,DNS,NaN,\n\n\n\n\n\nIndependent Submission ...,independent submission rye request comments be...
3,8553,DNS Attrleaf Changes: Fixing Specifications Th...,March 2019,BEST CURRENT PRACTICE,IETF,ops,dnsop,DNS,"DNS, Domain Name System",\n\n\n\n\n\nInternet Engineering Task Force (I...,internet engineering task force ietf crocker r...
4,8552,Scoped Interpretation of DNS Resource Records ...,March 2019,BEST CURRENT PRACTICE,IETF,ops,dnsop,DNS,"DNS, Domain Name System",\n\n\n\n\n\nInternet Engineering Task Force (I...,internet engineering task force ietf crocker r...


In [38]:
n_features=2000
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df_dns_full['cutted_words'])
lda = LatentDirichletAllocation(n_components=7, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
class_result=lda.fit_transform(tf)


In [39]:
def dic_top_words(model, feature_names, n_top_words):
    t_dic={}
    for topic_idx, topic in enumerate(model.components_):
        t_dic[topic_idx]=" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
    print (t_dic) 
    return t_dic



n_top_words = 5
tf_feature_names = tf_vectorizer.get_feature_names()
topic_dic=dic_top_words(lda, tf_feature_names, n_top_words)






{0: 'arpa addr ptr ipv practice', 1: 'multicast discovery user services srv', 2: 'object metadata uri srv description', 3: 'client message option private nat', 4: 'rrset dnskey parent soa update', 5: 'informational label labels mail info', 6: 'nsec algorithm sig owner signature'}


In [40]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)


C:\Users\wuyim\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.091040  0.012025       1        1  23.454263
6      0.094944 -0.024359       2        1  19.223413
3     -0.015793  0.086988       3        1  14.805889
2     -0.074193 -0.041049       4        1  14.689261
5     -0.009216 -0.039257       5        1  11.842585
1     -0.050988  0.040355       6        1   8.992679
0     -0.035792 -0.034704       7        1   6.991910, topic_info=     Category         Freq           Term        Total  loglift  logprob
111   Default   733.000000           arpa   733.000000  30.0000  30.0000
1141  Default   600.000000      multicast   600.000000  29.0000  29.0000
1178  Default  1297.000000           nsec  1297.000000  28.0000  28.0000
37    Default   433.000000           addr   433.000000  27.0000  27.0000
1668  Default   794.000000            sig   794.000000  26.0000  26.0000
286   Default  1662.000000         client  1662.000000  25.0000  25.0000
59    Default  1130.000000      algorithm  1130.000000  24.0000  24.0000
1185  Default   475.000000         object   475.000000  23.0000  23.0000
1104  Default  1090.000000        message  1090.000000  22.0000  22.0000
522   Default   357.000000      discovery   357.000000  21.0000  21.0000
1724  Default   522.000000            srv   522.000000  20.0000  20.0000
1222  Default   665.000000         option   665.000000  19.0000  19.0000
1588  Default  1021.000000          rrset  1021.000000  18.0000  18.0000
926   Default   690.000000            ipv   690.000000  17.0000  17.0000
537   Default   980.000000         dnskey   980.000000  16.0000  16.0000
1108  Default   331.000000       metadata   331.000000  15.0000  15.0000
1406  Default   338.000000            ptr   338.000000  14.0000  14.0000
1150  Default   338.000000            nat   338.000000  13.0000  13.0000
1672  Default   733.000000      signature   733.000000  12.0000  12.0000
1655  Default   362.000000        session   362.000000  11.0000  11.0000
269   Default   513.000000          child   513.000000  10.0000  10.0000
1258  Default   608.000000         parent   608.000000   9.0000   9.0000
1045  Default   489.000000           mail   489.000000   8.0000   8.0000
1918  Default   425.000000            uri   425.000000   7.0000   7.0000
1246  Default   678.000000          owner   678.000000   6.0000   6.0000
949   Default   939.000000           keys   939.000000   5.0000   5.0000
535   Default   361.000000          dname   361.000000   4.0000   4.0000
1166  Default   539.000000           node   539.000000   3.0000   3.0000
565   Default   395.000000            edu   395.000000   2.0000   2.0000
561   Default   585.000000       eastlake   585.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
1107   Topic7    31.602821           meta    84.788112   1.6735  -6.2872
758    Topic7    38.786171            gov   104.957654   1.6649  -6.0823
1378   Topic7    27.328950        project    74.060159   1.6635  -6.4325
1144   Topic7    66.066076    nameservers   187.312738   1.6183  -5.5498
898    Topic7    39.583096            int   112.408604   1.6167  -6.0620
1766   Topic7    54.313414         subnet   154.705514   1.6137  -5.7456
643    Topic7    48.914813         expire   139.749610   1.6106  -5.8503
1817   Topic7    28.035981       template    80.506283   1.6056  -6.4069
86     Topic7    39.390600        anycast   114.866126   1.5902  -6.0669
1417   Topic7    12.541194         qtypes    37.192007   1.5733  -7.2114
1023   Topic7    81.278544       location   248.995618   1.5409  -5.3425
119    Topic7    54.417419     assignment   168.546895   1.5299  -5.7437
183    Topic7   112.587164           best   388.750577   1.4212  -5.0167
1143   Topic7    87.771971     nameserver   296.768338   1.4422  -5.2657
1331   Topic7   118.253359       practice   457.333535   1.3078  -4.9676
926    Topic7   137.799556            ip